In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark data cleaning and engineering") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
import sys, tempfile, urllib, os
from pyspark.sql.functions import *

In [3]:
base_dir = "datasets/"
corona_data_file = os.path.join(base_dir, "corona_data.csv")

In [4]:
corona_data = urllib.request.urlretrieve("https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv", corona_data_file)

In [5]:
base_dir = "datasets/"
twitter_data_file = os.path.join(base_dir, "twitter_data.csv")

In [6]:
tweet_data = urllib.request.urlretrieve("https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv", twitter_data_file)

In [7]:
corona_df = spark.read.option("inferschema", "true").csv("datasets/corona_data.csv", header=True)

In [8]:
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [9]:
corona_df.count()

28143

In [10]:
twitter_df = spark.read.option("inferschema", "true").csv("datasets/twitter_data.csv", header=True)

In [11]:
twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [12]:
twitter_df.count()

1000

In [13]:
twitter_df.filter("country='USA'")

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [14]:
twitter_df.filter("country='USA'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [15]:
twitter_df.filter("country='USA' and location like '%New%'")

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [16]:
twitter_df.filter("country='USA' and location like '%New%'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

In [17]:
tw_filter_df = twitter_df.filter("country='USA'")

In [18]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#126, geo#127, text#128, user#129, location#130, entities#131, sentiment#132, country#133]
+- *(1) Filter (isnotnull(country#133) AND (country#133 = USA))
   +- FileScan csv [_c0#126,geo#127,text#128,user#129,location#130,entities#131,sentiment#132,country#133] Batched: false, DataFilters: [isnotnull(country#133), (country#133 = USA)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Rahul Ghosh/Desktop/Spark-for-ML/datasets/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,USA)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [19]:
tw_filter_df = twitter_df.filter("country='USA' and location like '%New%'")
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#126, geo#127, text#128, user#129, location#130, entities#131, sentiment#132, country#133]
+- *(1) Filter (((isnotnull(country#133) AND isnotnull(location#130)) AND (country#133 = USA)) AND Contains(location#130, New))
   +- FileScan csv [_c0#126,geo#127,text#128,user#129,location#130,entities#131,sentiment#132,country#133] Batched: false, DataFilters: [isnotnull(country#133), isnotnull(location#130), (country#133 = USA), Contains(location#130, New)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Rahul Ghosh/Desktop/Spark-for-ML/datasets/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), IsNotNull(location), EqualTo(country,USA), StringContains(location,New)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [20]:
tw_filter_df = twitter_df.filter(col("location").startswith("N"))

In [21]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#126, geo#127, text#128, user#129, location#130, entities#131, sentiment#132, country#133]
+- *(1) Filter (isnotnull(location#130) AND StartsWith(location#130, N))
   +- FileScan csv [_c0#126,geo#127,text#128,user#129,location#130,entities#131,sentiment#132,country#133] Batched: false, DataFilters: [isnotnull(location#130), StartsWith(location#130, N)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Rahul Ghosh/Desktop/Spark-for-ML/datasets/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), StringStartsWith(location,N)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [22]:
twitter_df.first()

Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria')

In [23]:
twitter_df.take(5)

[Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria'),
 Row(_c0=1, geo=None, text='"BREAKING: ""this is disappointing', user=' but i took the test"". Arsenal\'s Mikel Arteta tests positive for #coronavirus', location='  -… "', entities='JerryfranksonJF', sentiment='Abuja, Nigeria', country='"[(""Arsenal\'s Mikel Arteta""'),
 Row(_c0=2, geo=None, text=' #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…', user='cek422', location='Pennsylvania, USA', entities='[]', sentiment="{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}", country='USA'),
 Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', locatio

In [24]:
twitter_df.select("text").show()

+--------------------+
|                text|
+--------------------+
|What is God sayin...|
|"BREAKING: ""this...|
| #Coronavirus tes...|
| Get ready for ma...|
| The #coronavirus...|
| COVID-19 update ...|
| It’s painful to ...|
| 📽️Friends, I wi...|
| Questions about ...|
|How they’re deali...|
| BREAKING: Democr...|
| “If we close dow...|
| I pity the poor ...|
| We’re the heck w...|
| I don't think sh...|
| Well written, so...|
| 1/2 CDC Director...|
| In all seriousne...|
| Wash your hands....|
| #CoronaVirusCana...|
+--------------------+
only showing top 20 rows



In [25]:
twitter_df.select("text", "user").show()

+--------------------+--------------------+
|                text|                user|
+--------------------+--------------------+
|What is God sayin...|          petodinice|
|"BREAKING: ""this...| but i took the t...|
| #Coronavirus tes...|              cek422|
| Get ready for ma...|        InfectiousDz|
| The #coronavirus...|          vic_gibson|
| COVID-19 update ...|      StewartNgilana|
| It’s painful to ...|           BWheatnyc|
| 📽️Friends, I wi...|             LorseaR|
| Questions about ...|         straightj23|
|How they’re deali...|       _______coolio|
| BREAKING: Democr...|      champagneaylin|
| “If we close dow...|       YorkLawLondon|
| I pity the poor ...|      BeesonMargaret|
| We’re the heck w...|      harrytiffanyiv|
| I don't think sh...|         grammyheath|
| Well written, so...|      barbara_ellena|
| 1/2 CDC Director...|               fatal|
| In all seriousne...|          pwjkmiller|
| Wash your hands....|        Mrrandy123RP|
| #CoronaVirusCana...|           

In [26]:
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [27]:
corona_df.filter("Country='US'").show()

+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|_c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
| 98|      Washington|     US|           47.4009|-121.4905|2020-01-22|        0|    0|        0|      Washington|      Washington|
| 99|        New York|     US|           42.1657| -74.9481|2020-01-22|        0|    0|        0|        New York|        New York|
|100|      California|     US|           36.1162|-119.6816|2020-01-22|        0|    0|        0|      California|      California|
|101|   Massachusetts|     US|           42.2302| -71.5301|2020-01-22|        0|    0|        0|   Massachusetts|   Massachusetts|
|102|Diamond Princess|     US|           35.4437|  139.638|2020-01-22|        0|   

In [28]:
corona_df.filter("Country='US'").sort(col("Date"), ascending=False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [29]:
corona_df.filter("Country='US'").orderBy(col("Date"), ascending=False).show() 

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [30]:
corona_df.filter("Country='US'").sortWithinPartitions([col("Date"), col("Confirmed")], ascending=False).show()

+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20|      585|    5|        0|      Illinois|      Illinois|
|27772|       Florida|     US|27.7663| -81.6868|2020-03-20|      563|   

In [31]:
corona_df.describe().show()

+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|      Date|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|     28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|      null|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575|   

In [32]:
corona_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [33]:
corona_df.filter("Confirmed>1000").sort(col("Confirmed")).show()

+-----+----------+-------+-------+---------+----------+---------+-----+---------+-------------+----------+
|  _c0|     State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered|state_cleaned|      City|
+-----+----------+-------+-------+---------+----------+---------+-----+---------+-------------+----------+
|11609|     Hunan|  China|27.6104| 111.7088|2020-02-15|     1001|    2|      425|        Hunan|      null|
|12086|     Hunan|  China|27.6104| 111.7088|2020-02-16|     1004|    3|      464|        Hunan|      null|
| 7792|  Zhejiang|  China|29.1832| 120.0934|2020-02-07|     1006|    0|      123|     Zhejiang|      null|
|12563|     Hunan|  China|27.6104| 111.7088|2020-02-17|     1006|    3|      498|        Hunan|      null|
|13040|     Hunan|  China|27.6104| 111.7088|2020-02-18|     1007|    4|      527|        Hunan|      null|
|13517|     Hunan|  China|27.6104| 111.7088|2020-02-19|     1008|    4|      561|        Hunan|      null|
|13994|     Hunan|  China|27.6104| 11

In [34]:
corona_df.filter("Confirmed>10000").approxQuantile('Confirmed', [0.25, 0.5, 0.75, 0.9, 0.95], 0.9)

[10075.0, 10075.0, 10075.0, 10075.0, 67800.0]

In [35]:
corona_df.agg({"Date":"max"}).collect()

[Row(max(Date)='2020-03-20')]

In [36]:
corona_df.agg({"Date":"max", "Confirmed":"max"}).collect()

[Row(max(Confirmed)=67800, max(Date)='2020-03-20')]

In [37]:
max_date = corona_df.agg({"Date":"max"})

In [38]:
max_date.show()

+----------+
| max(Date)|
+----------+
|2020-03-20|
+----------+



In [41]:
import pyspark.sql.functions as F
corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date")).show()

+--------------+--------------------+----------+
|       Country|       State_cleaned| max(Date)|
+--------------+--------------------+----------+
|      Cameroon|             Yaounde|2020-03-20|
|         China|             Qinghai|2020-03-20|
|        Cyprus|             Nicosia|2020-03-20|
|            US|            Michigan|2020-03-20|
|      Portugal|              Lisbon|2020-03-20|
|            US|            Colorado|2020-03-20|
|United Kingdom|      Cayman Islands|2020-03-20|
|         China|              Hainan|2020-03-20|
|            US|            Missouri|2020-03-20|
|     Australia|Australian Capita...|2020-03-20|
|            US|                Guam|2020-03-20|
|        France|             Reunion|2020-03-20|
|      Colombia|        Cundinamarca|2020-03-20|
|          Cuba|              Havana|2020-03-20|
|     Mauritius|          Port Louis|2020-03-20|
|       Ukraine|                Kiev|2020-03-20|
|         Benin|          Porto-Novo|2020-03-20|
|   Switzerland|    

In [42]:
corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), 
               on=["Country", "State_cleaned", "Date"], how="inner").show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [43]:
corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), 
               on=["Country", "State_cleaned", "Date"], how="inner").sort("Confirmed", ascending=False).show(10)

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|    City|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|         China|           Hubei|2020-03-20|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|    null|
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|    null|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|    null|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|    null|
|          Iran|          Tehran|2020-03-20|27821|          null|   32.0|               53.0|    

In [44]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

ws = Window().partitionBy("Country", "State_cleaned").orderBy(col("Date").desc())
corona_df.withColumn("row_num", row_number().over(ws)).show()

+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|  _c0|State| Country|               Lat|   Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|27745| null|Cameroon|3.8480000000000003|11.5021|2020-03-20|       20|    0|        0|      Yaounde|null|      1|
|27268| null|Cameroon|3.8480000000000003|11.5021|2020-03-19|       13|    0|        0|      Yaounde|null|      2|
|26791| null|Cameroon|3.8480000000000003|11.5021|2020-03-18|       10|    0|        0|      Yaounde|null|      3|
|26314| null|Cameroon|3.8480000000000003|11.5021|2020-03-17|       10|    0|        0|      Yaounde|null|      4|
|25837| null|Cameroon|3.8480000000000003|11.5021|2020-03-16|        4|    0|        0|      Yaounde|null|      5|
|25360| null|Cameroon|3.8480000000000003|11.5021|2020-03-15|        2|    0|        0|  

In [45]:
corona_df.withColumn("row_num", row_number().over(ws)).where(col("row_num") == 1).show()

+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|  _c0|               State|       Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|    City|row_num|
+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|27745|                null|      Cameroon|3.8480000000000003|  11.5021|2020-03-20|       20|    0|        0|             Yaounde|    null|      1|
|27859|             Qinghai|         China|           35.7452|  95.9956|2020-03-20|       18|    0|       18|             Qinghai|    null|      1|
|27762|                null|        Cyprus|           35.1264|  33.4299|2020-03-20|       67|    0|        0|             Nicosia|    null|      1|
|27812|            Michigan|            US|           43.3266| -84.5361|2020-03-20|      552|    3|        0|   

In [47]:
corona_max_df = corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), 
               on=["Country", "State_cleaned", "Date"], how="inner").sort("Confirmed", ascending=False)

In [48]:
corona_df.groupBy("Country").pivot("Date").agg(F.sum("Confirmed")).show()

+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|    Country|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|2020-02-19|2020-

In [49]:
corona_df.filter("Country='US'").crosstab("State", "Date").show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         State_Date|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|

In [51]:
corona_max_df.groupBy("Country").agg({"Confirmed":"sum"}).show()

+-----------+--------------+
|    Country|sum(Confirmed)|
+-----------+--------------+
|       Chad|             1|
|   Paraguay|            13|
|     Russia|           253|
|    Senegal|            38|
|     Sweden|          1639|
| Cabo Verde|             1|
|     Guyana|             7|
|Philippines|           230|
|   Djibouti|             1|
|  Singapore|           385|
|   Malaysia|          1030|
|       Fiji|             1|
|     Turkey|           359|
|       Iraq|           208|
|    Germany|         19848|
|   Cambodia|            51|
|Afghanistan|            24|
|     Jordan|            85|
|   Maldives|            13|
|     Rwanda|            17|
+-----------+--------------+
only showing top 20 rows



In [52]:
corona_max_df.groupBy("Country").agg({"Confirmed":"sum", "Recovered":"sum", "Death":"sum"}).orderBy("sum(Confirmed)", ascending=False).show()

+--------------+--------------+----------+--------------+
|       Country|sum(Recovered)|sum(Death)|sum(Confirmed)|
+--------------+--------------+----------+--------------+
|         China|         71266|      3253|         81250|
|         Italy|          4440|      4032|         47021|
|         Spain|          1588|      1043|         20410|
|       Germany|           180|        67|         19848|
|          Iran|          6745|      1433|         19644|
|            US|             0|       244|         19100|
|        France|            12|       450|         12726|
|  Korea, South|          1540|        94|          8652|
|   Switzerland|            15|        54|          5294|
|United Kingdom|            67|       178|          4014|
|   Netherlands|             2|       107|          3003|
|       Austria|             9|         6|          2388|
|       Belgium|             1|        37|          2257|
|        Norway|             1|         7|          1914|
|        Swede

In [53]:
corona_df.filter("Country='Italy'").sort("Date", ascending=False).show()

+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|  _c0|State|Country| Lat|Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|27682| null|  Italy|43.0|12.0|2020-03-20|    47021| 4032|     4440|         Rome|null|
|27205| null|  Italy|43.0|12.0|2020-03-19|    41035| 3405|     4440|         Rome|null|
|26728| null|  Italy|43.0|12.0|2020-03-18|    35713| 2978|     4025|         Rome|null|
|26251| null|  Italy|43.0|12.0|2020-03-17|    31506| 2503|     2941|         Rome|null|
|25774| null|  Italy|43.0|12.0|2020-03-16|    27980| 2158|     2749|         Rome|null|
|25297| null|  Italy|43.0|12.0|2020-03-15|    24747| 1809|     2335|         Rome|null|
|24820| null|  Italy|43.0|12.0|2020-03-14|    21157| 1441|     1966|         Rome|null|
|24343| null|  Italy|43.0|12.0|2020-03-13|    17660| 1266|     1439|         Rome|null|
|23866| null|  Italy|43.0|12.0|2

In [54]:
corona_max_df = corona_max_df.withColumn("Active", corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death)

In [57]:
corona_max_df.groupBy("Country").sum("Active").orderBy("sum(Active)", ascending=False).show()

+--------------+-----------+
|       Country|sum(Active)|
+--------------+-----------+
|         Italy|      38549|
|       Germany|      19601|
|            US|      18856|
|         Spain|      17779|
|        France|      12264|
|          Iran|      11466|
|  Korea, South|       7018|
|         China|       6731|
|   Switzerland|       5225|
|United Kingdom|       3769|
|   Netherlands|       2894|
|       Austria|       2373|
|       Belgium|       2219|
|        Norway|       1906|
|        Sweden|       1607|
|       Denmark|       1327|
|      Portugal|       1009|
|      Malaysia|        940|
|        Canada|        922|
|       Czechia|        829|
+--------------+-----------+
only showing top 20 rows

